
1 = mmsegmentation RGB mask label change to mmseg gray style png  
2 = 注意：标注是跟图像同样的形状 (H, W)，其中的像素值的范围是 `[0, num_classes - 1]`。  
3 = 示例：[0, 2-1]   
4 = VOC数据集  ==> RGB mask label change to mmseg gray style png   
5 = 背景作为一个类 =背景作为0 ；【注意：转labelme JSON ,需要用含背景的greymask】  
    gray_img = np.zeros(shape=(color_img.shape[0], color_img.shape[1]), dtype=np.uint8) #背景参与计算, 背景设为0   
    [  
        (0, 0, 0),      # grey =0   # background #背景不参与计算, 背景设为255  
        (181, 119, 53), # grey =1     # pitaya # 读取 RGB (53, 119, 181) ==> cv2 ：BGR (181, 119, 53)     
    ] )  
6 = 背景不作为一个类 =背景作为255 ；【注意：不含背景的greymask，用于VOC训练时不含backguround类别】  
    gray_img = 255 - np.zeros(shape=(color_img.shape[0], color_img.shape[1]), dtype=np.uint8) #背景不参与计算, 背景设为255   
    cmap = np.array(    
    [  
        # (0, 0, 0),      # background #背景不参与计算, 背景设为255  
        (181, 119, 53),   # grey =0 # pitaya # 读取 RGB (53, 119, 181) ==> cv2 ：BGR (181, 119, 53)     
    ] )  
7 = 执行05-07程序，用含background的greymask 【特别注意】   

In [1]:
from PIL import Image
import numpy as np
import os
import cv2
import time
from tqdm import tqdm

1== 含背景greymask转换 

In [2]:
# RGB mask == > mmsegmetation mask 
def color2gray(img_path, color_map, save_dir):
    # 读取图片
    color_img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    # print('像素值= ', np.unique(color_img) )
    # 计算时间
    t0 = time.time()
    gray_img = np.zeros(shape=(color_img.shape[0], color_img.shape[1]), dtype=np.uint8)
    for i in range(color_map.shape[0]):
        index = np.where(np.all(color_img == color_map[i], axis=-1))  # np.all true false
        # print(i,'\n',index,'\n',color_map.shape[0])
        gray_img[index] = i
    t1 = time.time()
    time_cost = round(t1 - t0, 3)
    # print(f"color2label  cost time {time_cost}")
    # 保存图片
    dir, name = os.path.split(img_path)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, name)
    cv2.imwrite(save_path, gray_img)

In [3]:
# VOC 数据集  RGB mask ==> mmseg_gray_style mask
# 你的colormap # 特别注意 cv2 == BGR 
def RGBmask_to_mmseg_graymaskd(img_dir, save_dir):
    cmap = np.array(  
        [
            (0, 0, 0),      # gray =0 # background
            (181, 119, 53), # gray =1  pitaya # 读取 RGB (53, 119, 181) ==> cv2 ：BGR (181, 119, 53)   
        ] )


    if not os.path.exists(img_dir):
        os.mkdir(img_dir)

    for img in os.listdir(img_dir):
        if not img.endswith((".png", ".jpg")):
            continue
        img_path = os.path.join(img_dir, img)
        color2gray(img_path, color_map=cmap, save_dir=save_dir)

    path_valid =  os.path.join(root_path_gray, target_folder, os.listdir(img_dir)[0])
    img = cv2.imread(path_valid)
    np.unique(img)

if __name__=='__main__':
    # 文件路径
    root_path_gray = 'IMG_AUG'
    target_folder  = 'SegmentationClass_AUG_mask_graymmseg'
    img_dir = os.path.join(root_path_gray, 'SegmentationClass_AUG_mask')
    save_dir = os.path.join(root_path_gray, target_folder)
    
    RGBmask_to_mmseg_graymaskd(img_dir, save_dir) # RGB mask ==> mmseg_gray_style mask

2== 不含背景greymask转换 

In [4]:
# RGB mask == > mmsegmetation mask == 不含background 
def color2gray_nobackground(img_path, color_map, save_dir):
    # 读取图片
    color_img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    # print('像素值= ', np.unique(color_img) )
    # 计算时间
    t0 = time.time()
    gray_img = 255 - np.zeros(shape=(color_img.shape[0], color_img.shape[1]), dtype=np.uint8) #背景不参与计算, 背景设为255 
    for i in range(color_map.shape[0]):
        index = np.where(np.all(color_img == color_map[i], axis=-1))  # np.all true false
        # print(i,'\n',index,'\n',color_map.shape[0]) 
        gray_img[index] = i
    t1 = time.time()
    time_cost = round(t1 - t0, 3)
    # print(f"color2label  cost time {time_cost}")
    # 保存图片
    dir, name = os.path.split(img_path)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, name)
    cv2.imwrite(save_path, gray_img)


In [5]:
# VOC 数据集  RGB mask ==> mmseg_gray_style mask nobackguround
# 你的colormap # 特别注意 cv2 == BGR 
def RGBmask_to_mmseg_graymask_nobackground(img_dir, save_dir):
    cmap = np.array(  
        [
            # (0, 0, 0),      # background #背景不参与计算, 背景设为255
            (181, 119, 53),   # gray = 0 # pitaya # 读取 RGB (53, 119, 181) ==> cv2 ：BGR (181, 119, 53)   
        ] )

    if not os.path.exists(img_dir):
        os.mkdir(img_dir)

    for img in tqdm(os.listdir(img_dir)):
        if not img.endswith((".png", ".jpg")):
            continue
        img_path = os.path.join(img_dir, img) 
        color2gray_nobackground(img_path, color_map=cmap, save_dir=save_dir) 

    path_valid =  os.path.join(root_path_gray, target_folder, os.listdir(img_dir)[0])
    img = cv2.imread(path_valid)
    np.unique(img)

if __name__=='__main__':
    # 文件路径
    root_path_gray = 'IMG_AUG'
    target_folder  = 'SegmentationClass_AUG_mask_graymmseg_nobackground' # 无背景mask
    img_dir = os.path.join(root_path_gray, 'SegmentationClass_AUG_mask')
    save_dir = os.path.join(root_path_gray, target_folder)
    
    RGBmask_to_mmseg_graymask_nobackground(img_dir, save_dir) # RGB mask ==> mmseg_gray_style mask nobackguround 

100%|██████████| 42/42 [00:01<00:00, 34.79it/s]
